In [43]:
import pyopencl as cl
import time
import numpy as np

values = 1000000

# s_t = time.time()
def main_gpu() :
    cntxt = cl.create_some_context()
    queue = cl.CommandQueue(cntxt)

    n1 = np.array(range(values), dtype = np.int32)
    n2 = np.array(range(values), dtype = np.int32)
    out = np.empty(n1.shape, dtype = np.int32)

    n1_buffer = cl.Buffer(cntxt, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,hostbuf = n1)
    n2_buffer = cl.Buffer(cntxt, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,hostbuf = n2)

    out_buf = cl.Buffer(cntxt, cl.mem_flags.WRITE_ONLY, out.nbytes)

    code = """
    __kernel void frst_prog(__global int* num1, __global int* num2, __global int* out)
    {
        int i = get_global_id(0);
        out[i] = num1[i]*num1[i] + num2[i]*num2[i];
    }
    """

    bld = cl.Program(cntxt, code).build()
    launch = bld.frst_prog(queue,n1.shape, None,n1_buffer,n2_buffer,out_buf)
    launch.wait()

    cl.enqueue_copy(queue, out, out_buf)
# print(n1)
# print(n2)
# print(out)

# t = round((time.time() - s_t) * 1000)
# print("Time Taken : ", t, " ms")

In [44]:
def main_cpu() :
    n1 = np.array(range(values), dtype = np.int32)
    n2 = np.array(range(values), dtype = np.int32)
    out = np.empty(n1.shape, dtype = np.int32)
    for n in range(len(n1)) :
        out[n] = n1[n]*n1[n] + n2[n]*n2[n]



# s_t = time.time()


# # print(n1)
# # print(n2)
# # print(out)
# t = round((time.time() - s_t) * 1000)
# print("Time Taken : ", t, " ms")

In [45]:
%timeit main_gpu()
%timeit main_cpu()

466 ms ± 41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in int_scalars
  


5.21 s ± 73.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
